In [ ]:
import openai

- Explicit-Explicit Discovery: predict Arg2 sent-initial connective
- Explicit-Explicit PDTB: predict subsequent connective
- Explicit-Implicit: predict implicit 

In [ ]:
introduce_marker_symbol = '''\n  - Take note of the existing discourse marker enclosed by the character [ and ]'''

def prepare_system_prompt(type):
    prompt_file = ""
    if type == "free-insert":
        prompt_file = "free-insert_system_prompt.txt"
    elif type == "free-insert-co-occur":
        prompt_file = "free-insert-coo_system_prompt.txt"

    elif type == "mask-fill" or type == "mask-fill-with-marker-symbol":
        prompt_file = "mask-fill_system_prompt.txt"

    res = ""
    with open(prompt_file) as f:
        res = f.read()
    if "with-marker-symbol" in type:
        res = res.replace("<introduce_marker_symbol>", introduce_marker_symbol).replace("are curly and ___ are more", "are curly [and] ___ are more")
    else:
        res = res.replace("<introduce_marker_symbol>", "")
    return res

def prepare_user_prompt(arg1, conn1, arg2, type):
    res = ""
    with open("user_prompt.txt") as f:
        res = f.read()

    should_capitalize = arg1.rstrip().endswith(('.', '!', '?'))
    conn1 = conn1.capitalize() if should_capitalize else conn1

    if type == "free-insert" or type == "free-insert-co-occur":
        res = res.replace("<passage>", arg1 + " [" + conn1 + "] " + arg2)
    elif type == "mask-fill-with-marker-symbol":
        res = res.replace("<passage>", arg1 + " [" + conn1 + "] ___ " + arg2)
    elif type == "mask-fill":
        res = res.replace("<passage>", arg1 + " " + conn1 + " ___ " + arg2)
    return res

client = openai.OpenAI(
    api_key="",

)
def get_response (arg1, conn1, arg2, type):

    response = client.chat.completions.create(
        model="gpt-4.1-2025-04-14", 
        messages = [
            {
                "role": "developer",
                "content": prepare_system_prompt(type)
            },
            {
                "role": "user",
                "content": prepare_user_prompt(arg1, conn1, arg2, type)
            }
        ]
    )
    return response.choices[0].message.content

In [25]:
arg1 = '''More accurately, it was a humongous waste of space with no purpose except, perhaps, intimidation; it made visitors feel as if they'd wandered into somewhere they didn't belong.'''
conn1 = "fortunately,"
arg2 = '''the hotel's management took the storm damage as an opportunity to redesign.'''

In [27]:
for type in ["mask-fill", "mask-fill-with-marker-symbol", "free-insert", "free-insert-co-occur"]:
    
    print(f"*************** {type} ***************")
    system_prompt = prepare_system_prompt(type)
    user_prompt = prepare_user_prompt(arg1, conn1, arg2, type)

    with open(f"samples/{type}.txt", "w") as f:
        f.write(system_prompt + "\n\n" + user_prompt)
    print(system_prompt + "\n\n" + user_prompt)
    print("\n")

*************** mask-fill ***************
# Identity

You are a fluent and pragmatic English-speaker. In a given passage, there will be a blank denoted by ___.

# Instructions

  - Read the passage between the characters ```
  - Determine the 5 most likely English discourse markers in place of the blank ___ that is most appropriate for the context; Discourse markers are words or phrases that express the relation between adjacent parts of the passage. NO DUPLICATES IN THE LIST
  - Your response should only contain the 5 discourse markers separated by comma. 

# Examples

```
More common chrysotile fibers are curly and ___ are more easily rejected by the body.
```

OUTPUT: 
therefore, consequently, as a result, thus, hence 

Now you try:
```
More accurately, it was a humongous waste of space with no purpose except, perhaps, intimidation; it made visitors feel as if they'd wandered into somewhere they didn't belong. Fortunately, ___ the hotel's management took the storm damage as an oppor